### PART 1 
Installing and Importing programs to read and parse html document

In [1]:
import requests

In [2]:
!pip install BeautifulSoup4
!pip install lxml

     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 4.7MB/s eta 0:00:01     |█████████████████████████████▏  | 5.2MB 4.7MB/s eta 0:00:01


In [3]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())


#### Using tag 'td' to find and split values. Assign values to the empty lists (i,e., code_list)

In [4]:
import pandas as pd
codes_list=[]
borough_list=[]
neighborhood_list=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        tag = tag.text
        tag = tag.replace('\n', '')
        codes_list.append(tag)
        
    if i == 2:
        tag = tag.text
        tag = tag.replace('\n', '')
        borough_list.append(tag)
    if i == 3: 
        tag = tag.text
        tag = tag.replace('\n', '')
        neighborhood_list.append(tag)
    i = i+1
    if i==4:
        i=1
#combining lists into dataframe
df = pd.DataFrame()
df['PostalCode']=codes_list
df['Borough']=borough_list 
df['Neighborhood']=neighborhood_list
#replacing / with ,
df=df.replace({'/':','},regex=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo..."


#### Dropping not assigned Borough and reseting index

In [5]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df.shape

(103, 3)

### PART 2

##### installing geocoder to get coordinates

In [6]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    click-7.1.1                |     pyh8c360ce_0          64 KB  conda-forge
    cryptography-2.8           |   py36h45558ae_2         628 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    urllib3-1.25.8             |   py36h9f0ad1d_1         160 KB  conda-forge
    ------------------------------------------------------------
                  

In [7]:
import geocoder

In [ ]:
# select each row of df 
latitude=[]
longitude=[]
for i in range(len(df)):
#initialize lat_lng_coords to none
    lat_lng_coords = None    
#loop until you get the coordinates
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{},Toronto, Ontario'.format(df['PostalCode'][i]))
        lat_lng_coords=g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
df['Latitude']=latitude
df['Longitude']=longitude
#df1['lat']=latitude.append(lat_lng_coords[0])
#df1['log']=longitude.append(lat_lng_coords[1])
    #latitude.append(lat_lng_coords[0])
    #longitude.append(lat_lng_coords[1])


In [16]:
print(df.shape)
df.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390


### PART 3

##### choosing Borough that contains word 'York' only

In [20]:
df_York=df[df['Borough'].str.contains('York')].reset_index(drop=True)
print(df_York.shape)
df_York.head(10)


(34, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
3,M3B,North York,Don Mills,43.748900,-79.357220
4,M4B,East York,"Parkview Hill , Woodbine Gardens",43.707193,-79.311529
5,M6B,North York,Glencairn,43.707279,-79.447500
6,M3C,North York,Don Mills,43.722143,-79.352023
7,M4C,East York,Woodbine Heights,43.689740,-79.308507
8,M6C,York,Humewood-Cedarvale,43.691730,-79.430013
9,M6E,York,Caledonia-Fairbanks,43.689118,-79.450650


In [18]:

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



##### Map of all neighborhoods from Borough that contains word 'York' only

In [19]:

# create map of Manhattan using latitude and longitude values
map_df_York=folium.Map(location=[43.7001114, -79.4162979], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_York['Latitude'], df_York['Longitude'], df_York['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_df_York)  
    
map_df_York

### get venues near the neighborhoods

In [23]:
# @hidden_cell
CLIENT_ID =  your Foursquare ID
CLIENT_SECRET = '# your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Number of unique venues

In [74]:
york_venues = getNearbyVenues(names=df_York['Neighborhood'],
                                   latitudes=df_York['Latitude'],
                                   longitudes=df_York['Longitude']
                                  )
print('\n There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

Parkwoods
Victoria Village
Lawrence Manor , Lawrence Heights
Don Mills
Parkview Hill , Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Bayview Village
Downsview
York Mills , Silver Hills
Downsview
North Park , Maple Leaf Park , Upwood Park
Humber Summit
Willowdale , Newtonbrook
Downsview
Bedford Park , Lawrence Manor East
Del Ray , Mount Dennis , Keelsdale and Silverthorn
Humberlea , Emery
Willowdale
Downsview
Runnymede , The Junction North
Weston
York Mills West
Willowdale

 There are 132 uniques categories.


#### Getting the top 10 venues for each neighborhood

In [75]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()


,Neighborhood,Airport,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Baby Store,Bagel Shop,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped

,Neighborhood,Airport,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Baby Store,Bagel Shop,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,"Bathurst Manor , Wilson Heights , Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000
2,"Bedford Park , Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000
4,"Del Ray , Mount Dennis , Keelsdale and Silvert...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Don Mills,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.111111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Downsview,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.034483,0.000000,0.000000
7,"Fairview , Henry Farm , Oriole",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.038462,...,0.019231,0.000000,0.019231,0.019231,0.000000,0.019231,0.00,0.000000,0.038462,0.000000
8,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Supermarket,Chinese Restaurant,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Diner,Restaurant
1,Bayview Village,Construction & Landscaping,Flower Shop,Trail,Yoga Studio,Diner,Dog Run,Doctor's Office,Distribution Center,Discount Store,Department Store
2,"Bedford Park , Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Comfort Food Restaurant,Pharmacy,Pizza Place,Café,Butcher,Pub,Restaurant
3,Caledonia-Fairbanks,Park,Gym,Beer Store,Mexican Restaurant,Bakery,Sporting Goods Shop,Women's Store,Deli / Bodega,Department Store,Cosmetics Shop
4,"Del Ray , Mount Dennis , Keelsdale and Silvert...",Construction & Landscaping,Coffee Shop,Playground,Falafel Restaurant,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


#### Clustering Neighborhoods using 3 clusters

In [79]:
from sklearn.cluster import KMeans 
# set number of clusters
kclusters = 3

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 0, 2, 1, 1, 1, 1, 0], dtype=int32)

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = df_York

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

york_merged.head(10) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,0,Food & Drink Shop,Park,Yoga Studio,Dessert Shop,Dog Run,Doctor's Office,Distribution Center,Discount Store,Diner,Department Store
1,M4A,North York,Victoria Village,43.728102,-79.311890,1,Coffee Shop,Portuguese Restaurant,Pizza Place,Park,Intersection,Dessert Shop,Dog Run,Doctor's Office,Distribution Center,Discount Store
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211,1,Clothing Store,Restaurant,Cosmetics Shop,Bookstore,Sushi Restaurant,Men's Store,Food Court,Furniture / Home Store,American Restaurant,Chocolate Shop
3,M3B,North York,Don Mills,43.748900,-79.357220,1,Athletics & Sports,Coffee Shop,Park,Burger Joint,Restaurant,Other Great Outdoors,Bank,Cosmetics Shop,Deli / Bodega,Electronics Store
6,M3C,North York,Don Mills,43.722143,-79.352023,1,Athletics & Sports,Coffee Shop,Park,Burger Joint,Restaurant,Other Great Outdoors,Bank,Cosmetics Shop,Deli / Bodega,Electronics Store
4,M4B,East York,"Parkview Hill , Woodbine Gardens",43.707193,-79.311529,1,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Bank,Bus Line,Pharmacy,Breakfast Spot,Café,Intersection,Rock Climbing Spot
5,M6B,North York,Glencairn,43.707279,-79.447500,1,Pizza Place,Italian Restaurant,Pub,Rental Car Location,Mediterranean Restaurant,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
7,M4C,East York,Woodbine Heights,43.689740,-79.308507,1,Grocery Store,Bar,Pizza Place,Pharmacy,Pet Store,Music Store,Doctor's Office,Sushi Restaurant,Breakfast Spot,Coffee Shop
8,M6C,York,Humewood-Cedarvale,43.691730,-79.430013,0,Hockey Arena,Playground,Grocery Store,Field,Trail,Park,Business Service,Dessert Shop,Distribution Center,Discount Store
9,M6E,York,Caledonia-Fairbanks,43.689118,-79.450650,0,Park,Gym,Beer Store,Mexican Restaurant,Bakery,Sporting Goods Shop,Women's Store,Deli / Bodega,Department Store,Cosmetics Shop


####  Visualize the resulting clusters

In [84]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.7001114, -79.4162979], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0 red color. Neigborhoods like Parks

In [68]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[2] + list(range(5, york_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,0,Food & Drink Shop,Park,Yoga Studio,Dessert Shop,Dog Run,Doctor's Office,Distribution Center,Discount Store,Diner,Department Store
8,Humewood-Cedarvale,0,Hockey Arena,Playground,Grocery Store,Field,Trail,Park,Business Service,Dessert Shop,Distribution Center,Discount Store
9,Caledonia-Fairbanks,0,Park,Gym,Beer Store,Mexican Restaurant,Bakery,Sporting Goods Shop,Women's Store,Deli / Bodega,Department Store,Cosmetics Shop
11,Hillcrest Village,0,Residential Building (Apartment / Condo),Dog Run,Park,Department Store,Doctor's Office,Distribution Center,Discount Store,Diner,Dessert Shop,Yoga Studio
21,"North Park , Maple Leaf Park , Upwood Park",0,Massage Studio,Park,Basketball Court,Bakery,Yoga Studio,Discount Store,Electronics Store,Dog Run,Doctor's Office,Distribution Center
27,"Humberlea , Emery",0,Playground,Park,Falafel Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
32,York Mills West,0,Convenience Store,Park,Bank,Yoga Studio,Diner,Electronics Store,Dog Run,Doctor's Office,Distribution Center,Discount Store


#### Cluster 1 red color. Neigborhoods like Coffe Shops

In [85]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[2] + list(range(5, york_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,1,Coffee Shop,Portuguese Restaurant,Pizza Place,Park,Intersection,Dessert Shop,Dog Run,Doctor's Office,Distribution Center,Discount Store
2,"Lawrence Manor , Lawrence Heights",1,Clothing Store,Restaurant,Cosmetics Shop,Bookstore,Sushi Restaurant,Men's Store,Food Court,Furniture / Home Store,American Restaurant,Chocolate Shop
3,Don Mills,1,Athletics & Sports,Coffee Shop,Park,Burger Joint,Restaurant,Other Great Outdoors,Bank,Cosmetics Shop,Deli / Bodega,Electronics Store
6,Don Mills,1,Athletics & Sports,Coffee Shop,Park,Burger Joint,Restaurant,Other Great Outdoors,Bank,Cosmetics Shop,Deli / Bodega,Electronics Store
4,"Parkview Hill , Woodbine Gardens",1,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Bank,Bus Line,Pharmacy,Breakfast Spot,Café,Intersection,Rock Climbing Spot
5,Glencairn,1,Pizza Place,Italian Restaurant,Pub,Rental Car Location,Mediterranean Restaurant,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
7,Woodbine Heights,1,Grocery Store,Bar,Pizza Place,Pharmacy,Pet Store,Music Store,Doctor's Office,Sushi Restaurant,Breakfast Spot,Coffee Shop
10,Leaside,1,Sporting Goods Shop,Coffee Shop,Burger Joint,Sports Bar,Bank,Furniture / Home Store,Pet Store,Dessert Shop,Liquor Store,Portuguese Restaurant
12,"Bathurst Manor , Wilson Heights , Downsview North",1,Coffee Shop,Bank,Supermarket,Chinese Restaurant,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Diner,Restaurant
13,Thorncliffe Park,1,Indian Restaurant,Yoga Studio,Supermarket,Gas Station,Intersection,Middle Eastern Restaurant,Park,Coffee Shop,Sandwich Place,Bridge


#### Cluster 2 teal color. Neigborhoods like Construction & Landscaping

In [86]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[2] + list(range(5, york_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Bayview Village,2,Construction & Landscaping,Flower Shop,Trail,Yoga Studio,Diner,Dog Run,Doctor's Office,Distribution Center,Discount Store,Department Store
22,Humber Summit,2,Print Shop,Construction & Landscaping,Yoga Studio,Falafel Restaurant,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
26,"Del Ray , Mount Dennis , Keelsdale and Silvert...",2,Construction & Landscaping,Coffee Shop,Playground,Falafel Restaurant,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
